In [101]:
import math
from typing import List
from functools import lru_cache, reduce
from itertools import starmap, accumulate

import arrays
import trees
from importlib import reload
reload(arrays)
reload(trees)
import arrays as ar
import trees as tr
from trees import Node
print()

In [111]:

def to_linked_list(r):
    "Transform tree `r` into a double linked list in order."
    h = t = None
    for n in tr.df_nodes(r):
        h = h or n
        if t:
            t.right = n
        n.left, t = t, n
    if t:
        t.right = None
    return h

def dll_tree_it(n):
    while n:
        yield n.data
        n = n.right

In [112]:
list(dll_tree_it(to_linked_list(tr.make('2 1 -1 -1 3'))))

[1, 2, 3]